In [2]:
# !sudo apt-get update
# !sudo apt install tesseract-ocr
# !pip install pytesseract

In [3]:
pip install opencv-python-headless

In [4]:
wget https://github.com/oyyd/frozen_east_text_detection.pb/raw/master/frozen_east_text_detection.pb

SyntaxError: invalid syntax (2848011636.py, line 1)

In [5]:
import os
import time
import re
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pytesseract
from imutils.object_detection import non_max_suppression
from PIL import Image
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'imutils'

In [ ]:
def display_image(img):
    plt.figure(figsize=(10, 10))
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()

In [ ]:
from google.colab import files
uploaded = files.upload()
for filename in uploaded.keys():
  print('User uploaded file "{name}" with {length} bytes'.format(name=filename, length=len(uploaded[filename])))

Saving test_images.zip to test_images.zip
User uploaded file "test_images.zip" with 1304770 bytes


In [ ]:
zip_file_path = '/content/test_images.zip'
extraction_directory = '/content/test_images/'

!unzip "{zip_file_path}" -d "{extraction_directory}"

Archive:  /content/test_images.zip
  inflating: /content/test_images/img_annot.csv  
  inflating: /content/test_images/test_images/00bedd5c2fbf2dff.jpg  
  inflating: /content/test_images/test_images/00c7f84d61adce05.jpg  
  inflating: /content/test_images/test_images/03a1e7cf965c52c3.jpg  
  inflating: /content/test_images/test_images/0c2b594e8d66de4c.jpg  


In [ ]:
SAVED_MODEL_PATH = 'https://tfhub.dev/captain-pool/esrgan-tf2/1'

Masking Layer

In [ ]:
def post_process(orig, scores, geometry, confThreshold=0.5, nmsThreshold=0.4, rW=1, rH=1):
    (rects, confidences) = decode_predictions(scores, geometry, confThreshold)

    # Apply non-maxima suppression to suppress weak, overlapping bounding boxes
    boxes = non_max_suppression(np.array(rects), probs=confidences, overlapThresh=nmsThreshold)


    mask = np.zeros(orig.shape[:2], dtype='uint8')

    padding = 10  # Adjust padding as needed

    for (startX, startY, endX, endY) in boxes:
        # Apply padding around each bounding box and ensure the coordinates are within image bounds
        p_startX = max(0, int(startX * rW ) - padding )
        p_startY = max(0, int(startY * rH ) - padding)
        p_endX = min(orig.shape[1] -1 , int( endX * rW ) + padding )
        p_endY = min(orig.shape[0] -1 , int( endY * rH ) + padding )

        # Draw white rectangles on the mask for each bounding box (with padding)
        cv2.rectangle(mask, (p_startX, p_startY), (p_endX, p_endY), 255, -1)


    # Apply the mask to the original image
    masked_image = cv2.bitwise_and(orig, orig, mask=mask)

    return masked_image

In [ ]:
def decode_predictions(scores, geometry, confThreshold=0.5):
    # Grab the number of rows and columns from the scores volume, then
    # initialize our set of bounding box rectangles and corresponding
    # confidence scores
    (numRows, numCols) = scores.shape[2:4]
    rects = []
    confidences = []

    # Loop over the number of rows
    for y in range(0, numRows):
        # Extract the scores (probabilities), followed by the
        # geometrical data used to derive potential bounding box
        # coordinates that surround text
        scoresData = scores[0, 0, y]
        xData0 = geometry[0, 0, y]
        xData1 = geometry[0, 1, y]
        xData2 = geometry[0, 2, y]
        xData3 = geometry[0, 3, y]
        anglesData = geometry[0, 4, y]

        # Loop over the number of columns
        for x in range(0, numCols):
            # If our score does not have sufficient probability,
            # ignore it
            if scoresData[x] < confThreshold:
                continue

            # Compute the offset factor as our resulting feature
            # maps will be 4x smaller than the input image
            (offsetX, offsetY) = (x * 4.0, y * 4.0)

            # Extract the rotation angle for the prediction and
            # compute the sin and cosine
            angle = anglesData[x]
            cos = np.cos(angle)
            sin = np.sin(angle)

            # Use the geometry volume to derive the width and height
            # of the bounding box
            h = xData0[x] + xData2[x]
            w = xData1[x] + xData3[x]

            # Compute both the starting and ending (x, y)-coordinates
            # for the text prediction bounding box
            endX = int(offsetX + (cos * xData1[x]) + (sin * xData2[x]))
            endY = int(offsetY - (sin * xData1[x]) + (cos * xData2[x]))
            startX = int(endX - w)
            startY = int(endY - h)

            # Add the bounding box coordinates and probability score
            # to our respective lists
            rects.append((startX, startY, endX, endY))
            confidences.append(scoresData[x])

    # Return a tuple of the bounding boxes and associated confidences
    return (rects, confidences)


In [ ]:
##CNN Model for applying masking around the text
def detect_text_east(image_path):
    # Load image
    image = cv2.imread(image_path)
    orig = image.copy()
    (Ori_H, Ori_W) = image.shape[:2]

    # Set the new width and height and then determine the ratio in change
    (newW, newH) = (320, 320)
    rW = Ori_W / float(newW)
    rH = Ori_H / float(newH)

    # Resize the image and grab the new image dimensions
    image = cv2.resize(image, (newW, newH))
    (H, W) = image.shape[:2]

    # Define the two output layer names for the EAST detector model
    layerNames = [
        "feature_fusion/Conv_7/Sigmoid",
        "feature_fusion/concat_3"
    ]

    # Load the pre-trained EAST text detector
    net = cv2.dnn.readNet("frozen_east_text_detection.pb")

    # Construct a blob from the image and then perform a forward pass
    blob = cv2.dnn.blobFromImage(image, 1.0, (W, H),
                                 (123.68, 116.78, 103.94), swapRB=True, crop=False)
    net.setInput(blob)
    (scores, geometry) = net.forward(layerNames)

    rW = Ori_W / float(newW)
    rH = Ori_H / float(newH)

    # Decode the predictions, then  apply non-maxima suppression to suppress weak, overlapping bounding boxes
    result_image = post_process(orig.copy(), scores, geometry, confThreshold=0.5, nmsThreshold=0.4, rW=rW, rH=rH)
    # Return the original image (for now)
    return result_image



In [ ]:
# output_path = '/content/img_stage_1.jpg'
# cv2.imwrite(output_path, result_image)

In [ ]:
#IMAGE_PATH = '/content/img_stage_1.jpg'
#IMAGE_PATH = '/content/test_images/00bedd5c2fbf2dff.jpg'


Super Sampling Layer

In [ ]:
def preprocess_image(image_path):
    """Loads image from path and preprocesses to make it model ready
        :image_path: Path to the image file
    """
    hr_image = tf.image.decode_image(tf.io.read_file(image_path))
    # If PNG, remove the alpha channel. The model only supports
    # images with 3 color channels.
    if hr_image.shape[-1] == 4:
        hr_image = hr_image[...,:-1]
    hr_size = (tf.convert_to_tensor(hr_image.shape[:-1]) // 4) * 4
    hr_image = tf.image.crop_to_bounding_box(hr_image, 0, 0, hr_size[0], hr_size[1])
    hr_image = tf.cast(hr_image, tf.float32)
    return tf.expand_dims(hr_image, 0)

def save_image(image, filename):
    """
    Saves unscaled Tensor Images.
    :image: 3D image tensor. [height, width, channels]
    :filename: Name of the file to save.
    """
    if not isinstance(image, Image.Image):
        image = tf.clip_by_value(image, 0, 255)
        image = Image.fromarray(tf.cast(image, tf.uint8).numpy())
    image.save(filename)

In [ ]:
%matplotlib inline
def plot_image(image, title=""):
    """Plots images from image tensors.
      :image: 3D image tensor. [height, width, channels].
      :title: Title to display in the plot.
    """
    image = np.asarray(image)
    image = tf.clip_by_value(image, 0, 255)
    image = Image.fromarray(tf.cast(image, tf.uint8).numpy())
    plt.imshow(image)
    plt.axis("off")
    plt.title(title)

In [ ]:
model = hub.load(SAVED_MODEL_PATH)

In [ ]:
folder_path = '/content/test_images/test_images/'

Create output folder

In [ ]:
os.mkdir('/content/output/')

In [ ]:
pipeline_dict = {}

In [ ]:
os.listdir(folder_path)

['00c7f84d61adce05.jpg',
 '0c2b594e8d66de4c.jpg',
 '00bedd5c2fbf2dff.jpg',
 '03a1e7cf965c52c3.jpg']

In [ ]:
for image_file_name in os.listdir(folder_path):
    image_name = image_file_name.split('.')[0]
    image_path = folder_path + image_file_name

    if image_name is None or image_name == '':
      continue

    print(f'IMAGE NAME: ', image_name)
    start = time.time()

    result_image = detect_text_east(image_path)
    #display_image(result_image)
    output_path = f'/content/output/{image_name}_stage_1.jpg'
    status = cv2.imwrite(output_path, result_image)

    ## SUPER RESOLUTION
    IMAGE_PATH = f'/content/output/{image_name}_stage_1.jpg'
    hr_image = preprocess_image(IMAGE_PATH)
    #plot_image(tf.squeeze(hr_image), title="Original Image")

    fake_image = model(hr_image)
    fake_image = tf.squeeze(fake_image)

    #plot_image(tf.squeeze(fake_image), title="Super Resolution")
    save_image(tf.squeeze(fake_image), filename=f"/content/output/{image_name}_stage_2.jpg")

    ##OCR

    masked_image_path = f'/content/output/{image_name}_stage_2.jpg'
    #masked_image_path = '/content/test_images/00bedd5c2fbf2dff.jpg'

    # Read the image using OpenCV (PyTesseract also works with PIL images)
    masked_image = cv2.imread(masked_image_path)

    # Convert the image to RGB (PyTesseract expects images in RGB format)
    masked_image_rgb = cv2.cvtColor(masked_image, cv2.COLOR_BGR2RGB)

    # Use PyTesseract to extract text
    extracted_text = pytesseract.image_to_string(masked_image_rgb)

    words = extracted_text.split()
    words = [re.sub(r'[^\w\s]', '', word) for word in words]


    print("Time Taken: %f" % (time.time() - start))

    print('==================================')
    print("Extracted Text:")
    print(words)
    pipeline_dict[image_name] = words
    print('===================================')


IMAGE NAME:  00c7f84d61adce05
Time Taken: 24.691547
Extracted Text:
['fr', 'ret', 'Ef', 'ht']
IMAGE NAME:  0c2b594e8d66de4c
Time Taken: 27.638587
Extracted Text:
['', 'Bertolt', 'Brecht', '', 'Der', 'kaukasischd', 'Kreidekreis', '', 'edition', 'suhrkamp']
IMAGE NAME:  00bedd5c2fbf2dff
Time Taken: 19.321335
Extracted Text:
['E', 'TALK', 'IS', 'CHEAP', 'REALLY']
IMAGE NAME:  03a1e7cf965c52c3
Time Taken: 17.512483
Extracted Text:
['1assic', 'Sinat', 'ra', 'SEI', 'li', 'ae', '', '', 'Get', 'a', 'Kick', 'Out', 'of', 'hoy', 'They', 'Cant', 'Take', 'That', 'NEA', 'g', 'You', 'Make', 'Me', 'Feel', 'So', 'Young', 'Oey', 'iy', 'Fly', 'babi', 'Wit']


In [ ]:
#!rm -r /content/output/

In [ ]:
tesseract_dict = {}

In [ ]:
for image_file_name in os.listdir(folder_path):
    start = time.time()

    image_name = image_file_name.split('.')[0]
    image_path = folder_path + image_file_name

    print(f'Image Name: {image_name}')

    if image_name is None or image_name == '':
      continue
    masked_image_path = image_path
    #masked_image_path = '/content/test_images/00bedd5c2fbf2dff.jpg'

    # Read the image using OpenCV (PyTesseract also works with PIL images)
    masked_image = cv2.imread(masked_image_path)

    # Convert the image to RGB (PyTesseract expects images in RGB format)
    masked_image_rgb = cv2.cvtColor(masked_image, cv2.COLOR_BGR2RGB)

    # Use PyTesseract to extract text
    extracted_text = pytesseract.image_to_string(masked_image_rgb)

    words = extracted_text.split()
    words = [re.sub(r'[^\w\s]', '', word) for word in words]

    print("Time Taken: %f" % (time.time() - start))

    print('==================================')
    print("Extracted Text:")
    print(words)
    print('===================================')

    tesseract_dict[image_name] = words

Image Name: 00c7f84d61adce05
Time Taken: 0.724563
Extracted Text:
[]
Image Name: 0c2b594e8d66de4c
Time Taken: 0.787491
Extracted Text:
['Bertolt', 'Brecht', 'Der', 'kaukasische', 'Kreidekreis', 'edition', 'suhrkamp', 'SV', 'sea', '2S']
Image Name: 00bedd5c2fbf2dff
Time Taken: 0.540817
Extracted Text:
['', 'VACANCY']
Image Name: 03a1e7cf965c52c3
Time Taken: 0.695706
Extracted Text:
['Classic', 'Sinatra', 'Come', 'Fly', 'With', 'Me', 'Petes', 'Ue', 'eee', 'eae', 'ace', 'Esty', 'SONY', '', 'le', 'ETS', 'aR', 'ETT', 'ae', 'His', '', 'TTS', 'Wis', 'meek', 'Te', 'They', 'Cant', 'Take', 'That', 'Away', 'fr', 'You', 'Make', 'Me', 'Feel', 'So', 'ome', 'FY', 'Wa']
